In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pythainlp
import nltk
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

In [53]:
# import data
df = pd.read_csv('../Description_classification/Product_description.csv')
df.describe()

,Category
count,61.000000
mean,6.475410
std,4.064507
min,1.000000
25%,3.000000
50%,6.000000
75%,10.000000
max,13.000000


In [54]:
df.head()

,Product,Description,Category
0,INOVO,INOVO I 812 V1 (Ram:3GB/Rom:32GB) รับประกัน 1 ...,1
1,Pinno,"ขนาดหน้าจอ 2.4 นิ้ว,ระบบ Dual SIM (2 ซิม),รองร...",1
2,R9,มือถือจอหยดน้ำหน้าจอใหญ่สุดและสเปคแท้คุ้มค่าที...,1
3,J5pro (OEM),สินค้าลดล้างสต๊อกประกัน 7 วันเท่านั้น ไม่รองรั...,1
4,J9 Plus,J9 Plusมือถือ OEM ตอบโจทย์ทางเลือกสำหรับงบจำกั...,1


In [55]:
x = df['Description'].values
y = df['Category'].values

In [56]:
x[0]

'INOVO I 812 V1 (Ram:3GB/Rom:32GB) รับประกัน 1 ปี ,จอ 6.0" QHD  Quad-Core Processor Ram 3 GB/Rom 32 GB แบต 2500 mAh Battery ,5.0 MP+8.0MP Pixel , Face uniock, Android 8'

In [57]:
import re
import string

def clean_msg(msg):
    
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

def split_word(text):
    words = word_tokenize(text,engine='attacut')

    # Remove stop words TH and EN
    words = [i for i in words if not i in th_stop and not i in en_stop]

    # รากศัพท์
    # EN
    words = [p_stemmer.stem(i) for i in words]
    
    # TH
    tokens_temp=[]
    for i in words:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0) and (len(w_syn[0].lemma_names('tha'))!=3):
            if(w_syn[0].lemma_names('tha')[0]=='แบต'):
                tokens_temp.append(w_syn[0].lemma_names('tha')[1])
            else:
                tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    words = tokens_temp
    
    # ลบช่องว่าง
    words = [i for i in words if not ' ' in i]

    return words

In [58]:
clean_des = [clean_msg(text) for text in x]

In [59]:
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [60]:
word_list = [split_word(txt) for txt in clean_des]
print(word_list)

[['inovo', 'I', '812', 'V', '1', 'ram', '3', 'gbrom', '32', 'GB', 'รับประกัน', '1', 'ปี', 'จอ', '60', 'qhd', 'quadcor', 'processor', 'ram', '3', 'gbrom', '32', 'GB', 'แบตเตอรี่', '2500', 'mah', 'batteri', '50', 'MP', '80', 'MP', 'pixel', 'face', 'uniock', 'android', '8'], ['ขนาด', 'หน้าจอ', '24', 'นิ้ว', 'ระบบ', 'dual', 'sim', '2', 'ซิม', 'ค้ำ', 'SD', 'card', '8', 'GB', 'ปุ่มกด', 'สะใจ', 'กด', 'เห็นชัด', 'ช่อง', 'สอด', 'หูฟัง', 'ขนาด', '35', 'mm', 'ฟัง', 'ค์ชั่น', 'เครื่อง', 'MP', '3', 'MP', '4', 'FM', 'กล้อง', 'อัดเทป', 'นาฬิกาปลุก', 'ไฟฉาย', 'ภาษา', 'ค้ำ', 'english', 'russian', 'french', 'spanish', 'indonesiamelayu', 'thai', 'ความถี่', 'dual', 'band', 'gsm', '900', '1800', 'mhz'], ['มือถือ', 'จอ', 'หยดน้ำ', 'หน้าจอ', 'สเปค', 'คุ้มค่า', 'ตลาด', 'ตอนนี้', 'หน้า', 'จอแสดงผล', '626', 'นิ้ว', 'ระบบปฏิบัติการ', 'android', 'ของแท้', 'ดี', 'สำหรับ', 'ธุรกรรม', 'การเงิน', 'ปลอดภัย', 'No', 'root', 'googl', 'play', 'protect', 'ป้องกัน', 'แอ', 'ป', 'ทำอันตราย', 'ซอ', 'ฟแวร์', 'อัพเดท', 'ลอยแพ', 

In [61]:
tokens_list_j = [','.join(tkn) for tkn in word_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [62]:
cvec.vocabulary_

{'inovo': 530,
 'I': 285,
 '812': 223,
 'V': 315,
 '1': 11,
 'ram': 687,
 '3': 106,
 'gbrom': 478,
 '32': 114,
 'GB': 279,
 'รับประกัน': 1410,
 'ปี': 1262,
 'จอ': 996,
 '60': 194,
 'qhd': 682,
 'quadcor': 684,
 'processor': 671,
 'แบตเตอรี่': 1842,
 '2500': 90,
 'mah': 578,
 'batteri': 356,
 '50': 161,
 'MP': 293,
 '80': 219,
 'pixel': 659,
 'face': 448,
 'uniock': 783,
 'android': 337,
 '8': 218,
 'ขนาด': 903,
 'หน้าจอ': 1580,
 '24': 81,
 'นิ้ว': 1197,
 'ระบบ': 1398,
 'dual': 428,
 'sim': 725,
 '2': 64,
 'ซิม': 1053,
 'ค้ำ': 989,
 'SD': 306,
 'card': 378,
 'ปุ่มกด': 1264,
 'สะใจ': 1523,
 'กด': 836,
 'เห็นชัด': 1805,
 'ช่อง': 1039,
 'สอด': 1514,
 'หูฟัง': 1612,
 '35': 124,
 'mm': 595,
 'ฟัง': 1325,
 'ค์ชั่น': 990,
 'เครื่อง': 1688,
 '4': 135,
 'FM': 277,
 'กล้อง': 861,
 'อัดเทป': 1641,
 'นาฬิกาปลุก': 1192,
 'ไฟฉาย': 1938,
 'ภาษา': 1338,
 'english': 439,
 'russian': 702,
 'french': 469,
 'spanish': 740,
 'indonesiamelayu': 527,
 'thai': 767,
 'ความถี่': 946,
 'band': 355,
 'gsm': 498,
 

In [64]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
t_feat = tvec.fit_transform(tokens_list_j)